In [ ]:
import pandas as pd
import numpy as np
import geopandas as gp
import geoplot
import seaborn

In [ ]:
import matplotlib.pyplot as plt
import time
import shapely
import rtree

In [ ]:
import sklearn
from sklearn.ensemble import HistGradientBoostingClassifier as GBT
from sklearn.metrics import roc_curve, auc, roc_auc_score, make_scorer
from sklearn.inspection import permutation_importance
#from sklearn.utils.fixes import loguniform
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_validate, GridSearchCV
from scipy.stats import randint,uniform,loguniform
from sklearn.inspection import PartialDependenceDisplay

In [ ]:
from joblib import dump, load
from copy import deepcopy

---
# Config

In [ ]:
## Control runs (labels)
controls = [1,2,3,4,5]
runs = ['vet'] + ['full'] + controls

# Load training and testing sets

In [ ]:
# Load the train/test splits used in training controls
X_train = {}
X_test = {}
y_train = {}
y_test = {}
for r in controls:
    X_train[r], X_test[r], y_train[r], y_test[r] = load('/Data/TB_Diagnostics/final_data_split_VetOnly_Control_'+str(r)+'.data')

In [ ]:
# Load the train/test splits used in training vet only run
r = 'vet'
X_train[r], X_test[r], y_train[r], y_test[r] = load('/Data/TB_Diagnostics/final_data_split_VetOnly.data')

In [ ]:
# Load the train/test splits used in training full model
r = 'full'
X_train[r], X_test[r], y_train[r], y_test[r] = load('/Data/TB_Diagnostics/final_data_split.data')

# Load original data

In [ ]:
## Load data
data = pd.read_csv('/Data/TB_Diagnostics/inputVars_VetOnly.csv',parse_dates=['dateOfTest'],dtype=float)

In [ ]:
# Get target feature (confirmed breakdowns) as binary class
data_y = data.confirmedBreakdown.to_numpy().astype(bool)

In [ ]:
# Get observed features
data_X = data.drop(columns=['confirmedBreakdown'])

In [ ]:
# Convert dates to float
data_X.dateOfTest = data_X.dateOfTest.astype(int).astype(float)
# Add Random features
data_X['rand'] = np.random.random_sample(len(data_X))

# Model scoring functions

In [ ]:
## Function: sensitivity(prediction,target)
# returns sensitivity of prediction vs. target
# Se = TP / (TP + FN)
def sensitivity(p,t):
    TP = (p&t).sum()
    FN = (~p&t).sum()
    return TP / (TP + FN)

## Function: specificity(prediction,target)
# returns specificity of prediction vs. target
# Sp = TN / (TN + FP)
def specificity(p,t):
    TN = (~p&~t).sum()
    FP = (p&~t).sum()
    return TN / (TN + FP)

### SICCT Test performance

In [ ]:
sicct = {}
Se_sicct = {}
Sp_sicct = {}
Acc_sicct = {}
specificity_threshold = {}

for r in runs:
    sicct[r] = X_test[r][:,1].astype(bool)
    ## Sensitivity
    Se_sicct[r] = sensitivity(sicct[r],y_test[r])
    ## Specificity
    Sp_sicct[r] = specificity(sicct[r],y_test[r])
    ## Accuracy
    Acc_sicct[r] = (sicct[r]==y_test[r]).sum() / len(y_test[r])
    # Set specificity threshold to level for SICCT-only prediction
    specificity_threshold[r] = Sp_sicct[r]

# Load models

In [ ]:
# Load cross-validated and fit model
model = {}
for r in controls:
    model[r] = load('/Data/TB_Diagnostics/final_model_VetOnly_Control_'+str(r)+'.model')
    
model['vet'] = load('/Data/TB_Diagnostics/final_model_VetOnly.model')
model['full'] = load('/Data/TB_Diagnostics/final_model.model')

# Evaluate performance

In [ ]:
## Model score on testing set: (score is metric set at training time)
for r in controls:
    print(model[r].score(X_test[r],y_test[r]))

In [ ]:
## Get test predictions for more detailed evaluation:
y_test_result = {}
y_score = {}
for r in runs:
    y_test_result[r] = model[r].predict(X_test[r])
    y_score[r] = model[r].decision_function(X_test[r])

In [ ]:
## Sensitivity
Se = {}
for r in controls:
    Se[r] = sensitivity(y_test_result[r],y_test[r])
    print(Se[r])

In [ ]:
## Specificity
Sp = {}
for r in controls:
    Sp[r] = specificity(y_test_result[r],y_test[r])
    print(Sp[r])

In [ ]:
## Accuracy
Acc = {}
for r in controls:
    Acc[r] = (y_test_result[r]==y_test[r]).sum() / len(y_test[r])
    print(Acc[r])

In [ ]:
np.mean(list(Acc.values()))

---
# ROC Curves

In [ ]:
fpr = {}
tpr = {}
roc_auc = {}
for r in controls:
    fpr[r], tpr[r], _ = roc_curve(y_test[r],y_score[r])
    roc_auc[r] = auc(fpr[r],tpr[r])
    print(roc_auc[r])

In [ ]:
np.mean(list(roc_auc.values()))

In [ ]:
#function ot plot roc curve
def plot_roc(fpr,tpr,roc_auc,run):
    r=run
    plt.figure()
    lw = 2
    plt.plot(
        fpr[r],
        tpr[r],
        lw=lw,
        label="Model (AUC = %0.2f)" % roc_auc[r],
    )
    plt.plot(1-Sp_sicct[r],Se_sicct[r],'+', label="SICCT only", ms='15')
    plt.plot([0, 1], [0, 1], lw=lw, linestyle="--", label='Random')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel("(1 - Specificity)")
    plt.ylabel("Sensitivity")
    plt.title("Receiver operating characteristic")
    plt.legend(loc="lower right")

In [ ]:
plt.rcParams.update({'font.size': 16})
for r in controls:
    plot_roc(fpr,tpr,roc_auc,r)
    #plt.savefig('../Paper/figs/roc.pdf',bbox_inches='tight')

---
# Feature importance

In [ ]:
%%time
## Calcuate permutation importance
importance = {}
for r in runs:
    importance[r] = permutation_importance(model[r],X_test[r],y_test[r], n_repeats=20, n_jobs=-1)

In [ ]:
#for i in importance.importances_mean.argsort()[::-1]:
#    if abs(importance.importances_mean[i]) - 2 * importance.importances_std[i] > 0:
#        print('*',f"{importance.importances_mean[i]:.5f}", f" +/- {importance.importances_std[i]:.5f}", data_X.columns[i])
#    else:
#        print(' ',f"{importance.importances_mean[i]:.5f}", f" +/- {importance.importances_std[i]:.5f}", data_X.columns[i])

In [ ]:
#transform into table
importance_table = {}
for r in runs:
    importance_table[r] = pd.DataFrame(importance[r].importances.T, columns=data_X.columns)

In [ ]:
mean_order = {}
mean_order_nozero = {}
for r in runs:
    mean_order[r] = list(importance_table[r].mean().sort_values(ascending=False).index)
    mean_order_nozero[r] = list(importance_table[r].mean()[importance_table[r].mean()>0].sort_values(ascending=False).index)

In [ ]:
#Define feature lables:
feature_labels = {'resultOfTest':'Herd-level SICCT result',
                 'locationY':'Holding location Easting',
                 'locationX':'Holding location Northing',
                 'daysSinceBreakdown':'Days since herd breakdown *',
                 'dateOfTest':'Date of herd SICCT testing',
                 'animalsTested':'Number of animals tested',
                 'daysSincePreviousTest':'Time since previous SICCT test in herd *',
                 'severe':'Was the severe interpretation applied?',
                 'previousTestResult':'Result of last previous SICCT test in herd *',
                 'previousTestResult2':'Result of 2nd last previous SICCT test in herd *',
                 'gammaTestCount':'Number of historical GammaIFN test events in herd',
                 'rand':'Set of uniformly distributed random numbers (CONTROL)',
                 'inflow1':'Animals moved into herd, 1 year',
                 'inflow2':'Animals moved into herd, 2 years',
                 'inflow4':'Animals moved into herd, 4 years',
                 'inflow90':'Animals moved into herd, 90 days',
                 'outflow1':'Animals moved out herd, 1 year',
                 'outflow2':'Animals moved out herd, 2 years',
                 'outflow4':'Animals moved out herd, 4 years',
                 'outflow90':'Animals moved out herd, 90 days',
                 'inflowBD1':'Animals moved into herd, 1 year, from recent breakdown herds',
                 'inflowBD2':'Animals moved into herd, 2 years, from recent breakdown herds',
                 'inflowBD4':'Animals moved into herd, 4 years, from recent breakdown herds',
                 'inflowBD90':'Animals moved into herd, 90 days, from recent breakdown herds',
                 'outflowBD1':'Animals moved out herd, 1 year, from recent breakdown herds',
                 'outflowBD2':'Animals moved out herd, 2 years, from recent breakdown herds',
                 'outflowBD4':'Animals moved out herd, 4 years, from recent breakdown herds',
                 'outflowBD90':'Animals moved out herd, 90 days, from recent breakdown herds',
                 'vetPractice':'Veterinary practice conducting the test **',
                 'batchBovine':'Tuberculin batch (bovine) **',
                 'batchAvian':'Tuberculin batch (avian) **',
                 'testType':'Type of testing event',
                 'herdSize':'Size of herd at time of test',
                 'herdType':'Herd type (dairy, beef, etc.)',
                 'monthOfTest':'Month in which test taken',
                 'defraRiskScore':'APHA risk score for herd',
                 'meanBadgerAbundance':'Mean badger abundance'}
def feature_label(x):
    try:
        return feature_labels[x]
    except KeyError:
        return x        

In [ ]:
# apply labels to mean ordered set
mean_order_labels = {}
mean_order_nozero_labels = {}
for r in runs:
    mean_order_labels[r] = list(map(lambda x:feature_label(x), mean_order[r]))
    mean_order_nozero_labels[r] = list(map(lambda x:feature_label(x), mean_order_nozero[r]))

In [ ]:
# aggregate all control runs:
importance_table_agg = pd.concat([importance_table[r] for r in controls],ignore_index=True)
mean_order_agg = list(importance_table_agg.mean().sort_values(ascending=False).index)
mean_order_labels_agg = list(map(lambda x:feature_label(x), mean_order_agg))

In [ ]:
## Plot aggregate rankings over all runs:

plt.rcParams.update({'font.size': 28})
#fig, ax = plt.subplots(figsize=(16,20))
fig, ax = plt.subplots(figsize=(16,20))
seaborn.barplot(importance_table_agg[mean_order_agg], orient='h', errorbar='ci', ax=ax)
ax.set_yticklabels(mean_order_labels_agg)
plt.title('Relative importance of model features (5 Control runs)')
#plt.xscale('log')
plt.savefig('../Paper/figs/importance-reduced-sample.pdf',bbox_inches='tight')

In [ ]:
importance_table_agg.mean()['vetPractice']*100

In [ ]:
## Plot for each run:

for r in runs:
    plt.rcParams.update({'font.size': 28})
    fig, ax = plt.subplots(figsize=(16,20))
    seaborn.barplot(importance_table[r][mean_order[r]], orient='h', errorbar='ci', ax=ax)
    ax.set_yticklabels(mean_order_labels[r])
    plt.title('Relative importance of model features')
    #plt.xscale('log')
    #plt.savefig('../Paper/figs/importance.pdf',bbox_inches='tight')

In [ ]:
for r in controls:
    print(importance_table[r].mean()['vetPractice']*100)

## Plot controls against vet only and full model:

In [ ]:
#label tables:
#importance_table['full']['model'] = 'full'
#importance_table['vet']['model'] = 'vet'
#importance_table_agg['model'] = 'control'

#aggregate
#importance_table_forPlot = pd.concat([importance_table['full'],importance_table['vet'],importance_table_agg])

In [ ]:
plt.rcParams.update({'font.size': 28})
fig, ax = plt.subplots(figsize=(16,20))
ord = mean_order['full']
seaborn.barplot(importance_table['full'][ord], orient='h', errorbar='ci', ax=ax, label='full', color='red', alpha=0.5)
#seaborn.barplot(importance_table['vet'][ord], orient='h', errorbar='ci', ax=ax, label='vet', color='blue', alpha=0.5)
ax.set_yticklabels(mean_order_labels['full'])
plt.title('Relative importance of model features (all runs)')
plt.xscale('log')
plt.legend()
#plt.savefig('../Paper/figs/importance.pdf',bbox_inches='tight')

In [ ]:
importance_table['full'].melt()

In [ ]:
# Rearange tables for plotting grouped bar plot for all three models (full, vet, controls):

order_by = 'vet' #order all by full model order
ord = mean_order[order_by] 
ord_labels = mean_order_labels[order_by]

t1 = importance_table['full'][ord].melt()
t1['model'] = 'full'
t2 = importance_table['vet'][ord].melt()
t2['model'] = 'vet'
t3 = importance_table_agg[ord].melt()
t3['model'] = 'control'

t_all = pd.concat([t1,t2,t3])

In [ ]:
# Plot all models in grouped bar plot:

plt.rcParams.update({'font.size': 28})
fig, ax = plt.subplots(figsize=(16,20))
seaborn.barplot(ax=ax, data=t_all, x='value', y='variable', hue='model', errorbar='ci')
ax.set_yticklabels(ord_labels)
plt.title('Relative importance of model features (all models)')
#plt.xscale('log')
#plt.legend()
plt.ylabel(None)
plt.xlabel('Relative Importance')
#plt.savefig('test.pdf',bbox_inches='tight')

---
# Decision threshold

In [ ]:
# function to apply decision threshold
def predict_with_threshold(X, model, decision_threshold):
    return model.predict_proba(X)[:,1]>=decision_threshold

In [ ]:
# try different thresholds
thresholds = np.linspace(0.0,1.0,101)
sens = np.zeros(len(thresholds)) #sensitivity at threshold
spec = np.zeros(len(thresholds)) #specificity at threshold
for x in range(len(thresholds)):
    y_th = predict_with_threshold(X_test,model,thresholds[x])
    sens[x] = sensitivity(y_th,y_test)
    spec[x] = specificity(y_th,y_test)

best_sens = max(sens[spec >= Sp_sicct]) #sensitivity s.t. specificity >= SICCT
best_thresh = min(thresholds[spec >= Sp_sicct]) #threshold with max sensitivity s.t. specificity >= SICCT

In [ ]:
plt.rcParams.update({'font.size': 16})
# function to plot thresholds
plt.plot(thresholds,sens,label='Model HSe')
plt.plot(thresholds,spec,label='Model HSp')
plt.xlim(0.2,0.8)
plt.ylim(0.5,1.0)
best_sens_label = 'Chosen HSe = '+str(round(best_sens*100,1))+'%'
sicct_sens_label = 'SICCT HSe = '+str(round(Se_sicct*100,1))+'%'
sicct_spec_label = 'SICCT HSp = '+str(round(Sp_sicct*100,1))+'%'
best_thresh_label = 'Chosen threshold = '+str(round(best_thresh,3))
plt.axvline(best_thresh,c='k',ls='-.',label=best_thresh_label)
plt.axhline(best_sens,c='k',ls='--',label=best_sens_label)
plt.axhline(Se_sicct,c='tab:blue',ls=':',label=sicct_sens_label)
plt.axhline(Sp_sicct,c='tab:orange',ls=':',label=sicct_spec_label)
plt.xlabel('Decision Threshold')
plt.legend(bbox_to_anchor=(1, 0.5))
#plt.savefig('../Paper/figs/decision_threshold.pdf',bbox_inches='tight')

In [ ]:
# Percentage increase in sensitivity over SICCT alone
(best_sens - Se_sicct)/Se_sicct*100

In [ ]:
#Predictions from model at threshold
y_test_predicted = predict_with_threshold(X_test, model, best_thresh)

### Test on 2020 only data

In [ ]:
# Get data for 2020 only
mask_2020 = data.dateOfTest.apply(lambda x:x.year)==2020
X_2020 = data_X[mask_2020].to_numpy()
y_2020 = data_y[mask_2020]

In [ ]:
#Predictions from model at threshold for 2020 only
y_2020_predicted = predict_with_threshold(X_2020, model, best_thresh)

### Test with HSp maximised

In [ ]:
# What if we maximise specificty instead?

sens_thresh = max(thresholds[sens>= Se_sicct]) # threshold with max specificity s.t. sensitivity >= SICCT
best_spec = max(spec[sens >= Se_sicct]) # specificty s.t. sensitivity >= SICCT

sens_thresh , best_spec

In [ ]:
#Predictions from model at HSp threshold
y_test_predicted_hsp = predict_with_threshold(X_test, model, sens_thresh)

---
# <mark>Plots

In [ ]:
#Projections:
bng = 'epsg:27700' # British National Grid
wgs84 = 'epsg:4326' # Lat.Long.

In [ ]:
#UK base map
uk_shp = gp.read_file('/Data/Shapefiles/bdline_essh_gb/Data/Supplementary_Country/country_region.shp').to_crs(wgs84)
#uk_shp.plot(color='white', edgecolor='black')

## Plot residuals

In [ ]:
residual = (y_test != y_test_predicted)

test_locs = pd.DataFrame({'locationX':X_test[:,5],'locationY':X_test[:,6],'date':X_test[:,0]})
test_locs.date = test_locs.date.astype('datetime64[ns]')

test_locs['residual'] = residual

test_geo = gp.GeoDataFrame(test_locs,geometry=gp.points_from_xy(test_locs.locationX,test_locs.locationY,crs=bng))
test_geo = test_geo.to_crs(wgs84)

errors = test_geo[test_geo.residual]

In [ ]:
# residuals for 2020 only
residual_2020 = (y_2020 != y_2020_predicted)

locs_2020 = pd.DataFrame({'locationX':X_2020[:,5],'locationY':X_2020[:,6],'date':X_2020[:,0]})
locs_2020.date = locs_2020.date.astype('datetime64[ns]')

locs_2020['residual'] = residual_2020

geo_2020 = gp.GeoDataFrame(locs_2020,geometry=gp.points_from_xy(locs_2020.locationX,locs_2020.locationY,crs=bng))
geo_2020 = geo_2020.to_crs(wgs84)

errors_2020 = geo_2020[geo_2020.residual]

In [ ]:
# Location of observations vs. residuals in Test set
ax = uk_shp.plot(alpha=0.2, figsize=(10,20))
#test_geo.plot('residual', markersize=1.0, ax=ax)
test_geo.plot(markersize=1.0, ax=ax)
errors.plot(markersize=1.0, color='red', ax=ax)

In [ ]:
# Location of observations vs. residuals in 2020 set
ax = uk_shp.plot(alpha=0.2, figsize=(10,20))
geo_2020.plot(markersize=1.0, ax=ax)
errors_2020.plot(markersize=1.0, color='red', ax=ax)

In [ ]:
# Residuals in time
test_times = pd.DataFrame({'date':X_test[:,0].copy().astype('datetime64[ns]')})
test_times['residual'] = residual
error_times = test_times[test_times.residual]
e = error_times.groupby(error_times["date"].dt.year).count().date
t = test_times.groupby(test_times["date"].dt.year).count().date
(e/t).plot.bar()
plt.title("Proportion of model misclassifications by year")
plt.xlabel('Year')
#plt.savefig('../Paper/figs/temporal.pdf',bbox_inches='tight')

## Plot newly discovered positives

In [ ]:
new_detected = (~X_test[:,1].astype(bool) & y_test_predicted)
test_geo['new_detected'] = new_detected

In [ ]:
# new detected in 2020
new_detected_2020 = (~X_2020[:,1].astype(bool) & y_2020_predicted)
geo_2020['new_detected'] = new_detected_2020

In [ ]:
# Location of observations vs. newly detected herds in Test set
ax = uk_shp.plot(alpha=0.2, figsize=(10,20))
test_geo.plot(markersize=1.0, ax=ax)
test_geo[test_geo.new_detected].plot(markersize=1.0, color='gold', ax=ax)

In [ ]:
ax = uk_shp.to_crs(bng).plot(alpha=0.2, figsize=(10,20))
seaborn.kdeplot(ax=ax, x=test_geo[test_geo.new_detected].locationX, y=test_geo[test_geo.new_detected].locationY, fill=True, color='gold')

## Plot Residuals and Newly Detected density by area

In [ ]:
## Create grid for normalising misclassifcation
# total area for the grid
xmin, ymin, xmax, ymax = uk_shp.to_crs(bng).total_bounds
#size of cell
cell_size = 10000 #10km x 10km squares
# create the cells in a loop
grid_cells = []
for x0 in np.arange(xmin, xmax+cell_size, cell_size ):
    for y0 in np.arange(ymin, ymax+cell_size, cell_size):
        # bounds
        x1 = x0-cell_size
        y1 = y0+cell_size
        grid_cells.append(shapely.geometry.box(x0, y0, x1, y1))
grid = gp.GeoDataFrame(grid_cells, columns=['geometry'], crs=bng)

In [ ]:
# For normalisation of map cells by number of tests in cell:
# number of tests in test set
grid_n_tests = gp.sjoin(test_geo.to_crs(bng), grid, how='left', predicate='within')
grid_n_tests['n_tests'] = 1
grid_n_tests_d = grid_n_tests.dissolve(by='index_right', aggfunc='count')
grid.loc[grid_n_tests_d.index, 'n_tests'] = grid_n_tests_d.n_tests.values

#number of tests in 2002
grid_n_tests_2020 = gp.sjoin(geo_2020.to_crs(bng), grid, how='left', predicate='within')
grid_n_tests_2020['n_tests20'] = 1
grid_n_tests_2020_d = grid_n_tests_2020.dissolve(by='index_right', aggfunc='count')
grid.loc[grid_n_tests_2020_d.index, 'n_tests20'] = grid_n_tests_2020_d.n_tests.values

In [ ]:
grid.plot(column='n_tests')
grid.plot(column='n_tests20')

In [ ]:
## Spatial join residuals with grid
## and plot to produce heatmap
errors_grid = gp.sjoin(errors.to_crs(bng), grid, how='left', predicate='within')

# Compute residuals per grid cell
errors_grid['n_resid'] = 1
#errors_grid_d = errors_grid.dissolve(by="index_right", aggfunc="count")
errors_grid_n_resid = errors_grid[['index_right','n_resid']].groupby(by="index_right").count()

# Add to grid
grid.loc[errors_grid_n_resid.index, 'n_resid'] = errors_grid_n_resid

# add normalised to grid
grid.loc[errors_grid_n_resid.index, 'norm_resid'] = grid.loc[errors_grid_n_resid.index, 'n_resid'] / grid.loc[errors_grid_n_resid.index, 'n_tests']

In [ ]:
# Plot errors grid
ax = grid.plot(column='n_resid', cmap='YlOrRd', legend=True, legend_kwds={'shrink': 0.3}, figsize=(10,20))
uk_shp.to_crs(bng).plot(ax=ax,alpha=0.1)
plt.title("Misclassified tests by area (in test set)")
plt.axis('off')
#plt.savefig('../Paper/figs/map_misclassified.pdf',bbox_inches='tight')

In [ ]:
# Plot errors grid (normalised)
ax = grid.plot(column='norm_resid', cmap='YlOrRd', legend=True, legend_kwds={'shrink': 0.3}, figsize=(10,20))
uk_shp.to_crs(bng).plot(ax=ax,alpha=0.1)
plt.title("Proportion of tests misclassified by area (in test set)")
plt.axis('off')
#plt.savefig('../Paper/figs/map_misclassified.pdf',bbox_inches='tight')

In [ ]:
## compute residuals grid for 2020 only
errors20_grid = gp.sjoin(errors_2020.to_crs(bng), grid, how='left', predicate='within')

# Compute residuals per grid cell
errors20_grid['n_resid20'] = 1
errors20_grid_n_resid = errors20_grid[['index_right','n_resid20']].groupby(by="index_right").count()

# Add to grid
grid.loc[errors20_grid_n_resid.index, 'n_resid20'] = errors20_grid_n_resid

# add normalised to grid
grid.loc[errors20_grid_n_resid.index, 'norm_resid20'] = grid.loc[errors20_grid_n_resid.index, 'n_resid20'] / grid.loc[errors20_grid_n_resid.index, 'n_tests20']

In [ ]:
# Plot errors grid for 2020 only
ax = grid.plot(column='n_resid20', cmap='YlOrRd', legend=True, legend_kwds={'shrink': 0.3}, figsize=(10,20))
uk_shp.to_crs(bng).plot(ax=ax,alpha=0.1)
plt.title("Misclassified tests by area (in 2020)")
plt.axis('off')
#plt.savefig('../Paper/figs/map_misclassified_2020.pdf',bbox_inches='tight')

In [ ]:
# Plot normalised errors grid for 2020 only
ax = grid.plot(column='norm_resid20', cmap='YlOrRd', legend=True, legend_kwds={'shrink': 0.3}, figsize=(10,20))
uk_shp.to_crs(bng).plot(ax=ax,alpha=0.1)
plt.title("Proportion of tests misclassified by area (in 2020)")
plt.axis('off')
#plt.savefig('../Paper/figs/map_misclassified_2020.pdf',bbox_inches='tight')
#plt.savefig('../Paper/figs/map_misclassified_2020.png',bbox_inches='tight')

In [ ]:
## Spatial join newly detected with grid
## and plot to produce heatmap
new_detect_grid = gp.sjoin(test_geo[test_geo.new_detected].to_crs(bng), grid, how='left', predicate='within')

# Compute new detects per grid cell -- aggregate with dissolve
new_detect_grid['n_new'] = 1
new_detect_grid_n_new = new_detect_grid[['index_right','n_new']].groupby(by="index_right").count()

# Add to grid
grid.loc[new_detect_grid_n_new.index, 'n_new'] = new_detect_grid_n_new

# add normalised to grid
grid.loc[new_detect_grid_n_new.index, 'norm_new'] = grid.loc[new_detect_grid_n_new.index, 'n_new'] / grid.loc[new_detect_grid_n_new.index, 'n_tests']

In [ ]:
# Plot new grid
ax = grid.plot(column='n_new', cmap='cividis', legend=True, legend_kwds={'shrink': 0.3}, figsize=(10,20))
uk_shp.to_crs(bng).plot(ax=ax,alpha=0.1)
plt.title("Early detected tests by area (in test set)")
plt.axis('off')
#plt.savefig('../Paper/figs/map_newly_detected.pdf',bbox_inches='tight')

In [ ]:
grid.n_new.sum()

In [ ]:
# Plot new grid, normalised
ax = grid.plot(column='norm_new', cmap='cividis', legend=True, legend_kwds={'shrink': 0.3}, figsize=(10,20))
uk_shp.to_crs(bng).plot(ax=ax,alpha=0.1)
plt.title("Proportion of tests early detected by area (in test set)")
plt.axis('off')
#plt.savefig('../Paper/figs/map_newly_detected.pdf',bbox_inches='tight')

In [ ]:
## Spatial join newly detected with grid in 2020 only
new_detect20_grid = gp.sjoin(geo_2020[geo_2020.new_detected].to_crs(bng), grid, how='left', predicate='within')

# Compute new detects per grid cell -- aggregate with dissolve
new_detect20_grid['n_new20'] = 1
new_detect20_grid_n_new = new_detect20_grid[['index_right','n_new20']].groupby(by="index_right").count()

# Add to grid
grid.loc[new_detect20_grid_n_new.index, 'n_new20'] = new_detect20_grid_n_new

# add normalised to grid
grid.loc[new_detect20_grid_n_new.index, 'norm_new20'] = grid.loc[new_detect20_grid_n_new.index, 'n_new20'] / grid.loc[new_detect20_grid_n_new.index, 'n_tests20']

In [ ]:
# Plot new grid for 2020 only
ax = grid.plot(column='n_new20', cmap='cividis', legend=True, legend_kwds={'shrink': 0.3}, figsize=(10,20))
uk_shp.to_crs(bng).plot(ax=ax,alpha=0.1)
plt.title("Early detected tests by area (in 2020)")
plt.axis('off')
#plt.savefig('../Paper/figs/map_newly_detected_2020.pdf',bbox_inches='tight')

In [ ]:
grid.n_new20.sum()

In [ ]:
# Plot new grid for 2020 only, normalised
ax = grid.plot(column='norm_new20', cmap='cividis', legend=True, legend_kwds={'shrink': 0.3}, figsize=(10,20))
uk_shp.to_crs(bng).plot(ax=ax,alpha=0.1)
plt.title("Proportion of tests early detected by area (in 2020)")
plt.axis('off')
#plt.savefig('../Paper/figs/map_newly_detected_2020.pdf',bbox_inches='tight')
#plt.savefig('../Paper/figs/map_newly_detected_2020.png',bbox_inches='tight')

## Partial dependence

In [ ]:
#### Takes ages and eats memory... ?!?!
#PartialDependenceDisplay.from_estimator(model, X_train, [4,5,(4,5)])

## Plot feature correlations

---
# <mark>Analysis

## What sort of herd is newly detected?

* See map above for spatial distribution.
* What else?

## Confusion matrix

* all stats w.r.t. sicct alone and Stanski

In [ ]:
np.array([['tp','tn'],['fp','fn']])

In [ ]:
# Function ot calculate confusion matrix
# p = predicted class
# t = actual class
def confusion_matrix(p,t):
    # True Positives
    TP = (p&t).sum()
    # True Negatives
    TN = (~p&~t).sum()
    # False Positives
    FP = (p&~t).sum()
    # False Negatives
    FN = (~p&t).sum()
    # return matrix (values and proportions)
    total = len(p)
    val_array = np.array([[TP,TN],[FP,FN]])
    prop_array = np.around(np.array([[TP/total, TN/total], [FP/total, FN/total]]) * 100, 1)
    return val_array , prop_array

In [ ]:
# confusion matrix for test set
cm_model = confusion_matrix(y_2020_predicted,y_2020)
cm_model

In [ ]:
sensitivity(y_2020_predicted,y_2020), specificity(y_2020_predicted,y_2020)

In [ ]:
# confusion matrix for SICCT
sicct_2020_predicted = X_2020[:,1]==1
cm_sicct = confusion_matrix(sicct_2020_predicted,y_2020)
cm_sicct

In [ ]:
sensitivity(sicct_2020_predicted,y_2020), specificity(sicct_2020_predicted,y_2020)

In [ ]:
# reduction in FNs / FPs from SICCT to model
m_fn = cm_model[0][1][1]
m_fp = cm_model[0][1][0]
s_fn = cm_sicct[0][1][1]
s_fp = cm_sicct[0][1][0]

print('FN reduction: ', (s_fn - m_fn)/s_fn, '\nFP reduction:', (s_fp - m_fp)/s_fp)

In [ ]:
print('2020 HSe increase: ',(sensitivity(y_2020_predicted,y_2020) - sensitivity(sicct_2020_predicted,y_2020))/sensitivity(sicct_2020_predicted,y_2020))

In [ ]:
# How many FPs caught by HSp maximisation threshold?
y_2020_predicted_hse = predict_with_threshold(X_2020, model, sens_thresh)
# TNs where test was P, vs. all TNs  (in 2020)
sum((X_2020[:,1] == 1) & (y_2020_predicted_hse == 0) & (y_2020 == 0)) , sum((y_2020_predicted_hse == 0) & (y_2020 == 0))

## Number of days to breakdown (distribution)

* for newly detected / vs sicct detected
* other? 

## Normalised spatial analysis

* residuals normalised by no. of tests
* new detections ---"---

---
# <mark> Other TODOs:


* Permutation importance with multicolinear features
* Gold standard period? 90-day or other? Test?
* Time / area split models
    - Fix fitting to existing model, or re-tune for each model?